<a href="https://colab.research.google.com/github/raynertjx/20242R0136COSE47402/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from datasets import load_dataset, DatasetDict
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import os
import torch

def get_latest_checkpoint(model_dir):
    # List all subdirectories in the model directory
    checkpoints = [d for d in os.listdir(model_dir) if d.startswith("checkpoint-")]
    # Sort by checkpoint number (numerical order)
    latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split('-')[-1]))[-1]
    return os.path.join(model_dir, latest_checkpoint)

# Step 1: Load and Preprocess RecipeNLG Dataset
def load_recipenlg():
    print("Loading RecipeNLG dataset...")
    # Load the dataset using Hugging Face Datasets
    path = "/content/drive/MyDrive/Colab Notebooks/"
    dataset = load_dataset("recipe_nlg", data_dir=path)

    # Preprocess: Combine title, ingredients, and directions into a single text input
    def preprocess_data(example):
        return {
            "text": f"Title: {example['title']} Ingredients: {example['ingredients']} Instructions: {example['directions']}"
        }

    dataset = dataset.map(preprocess_data, remove_columns=["title", "ingredients", "directions"])

    # Ensure exact sizes for training and testing
    total_size = len(dataset["train"])
    train_size = 8000
    test_size = 2000

    dataset_split = dataset["train"].train_test_split(
        test_size=test_size / total_size,  # Calculate test split ratio
        seed=42
    )

    dataset = DatasetDict({
        "train": dataset_split["train"].select(range(train_size)),  # Limit train split to 8000
        "test": dataset_split["test"].select(range(test_size)),    # Limit test split to 2000
    })

    print(f"Dataset loaded and reduced: {dataset}")
    return dataset

# Step 2: Fine-Tune Pre-Trained Model
def fine_tune_model(dataset, model_name="gpt2", output_dir="fine_tuned_model"):
    print(f"Fine-tuning {model_name} on RecipeNLG...")
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    # Tokenize the dataset
    def tokenize_function(examples):
        tokenized = tokenizer(
            examples["text"], truncation=True, padding="max_length", max_length=512
        )
        tokenized["labels"] = tokenized["input_ids"].copy()  # Add labels for loss computation
        return tokenized

    print("Tokenizing dataset...")
    tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=2)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,  # Moderate batch size
        num_train_epochs=3,  # Use only 1 epoch for initial testing
        save_strategy="epoch",
        logging_dir="./logs",
        logging_steps=10,
        weight_decay=0.01,
        push_to_hub=False,
        report_to="none",
        fp16=True
    )

    # Trainer for fine-tuning
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
    )

    trainer.train()
    print(f"Model fine-tuned and saved to {output_dir}.")
    return output_dir

# Step 3: Generate Recipes
def generate_recipe(model_path, prompt, max_length=200):
    print("Loading fine-tuned model for generation...")
    latest_checkpoint = get_latest_checkpoint(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained(latest_checkpoint)
    model = GPT2LMHeadModel.from_pretrained(latest_checkpoint)
    tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.8,
        repetition_penalty=1.5,  # Penalize repeated phrases
        top_k=50,  # Use top-k sampling
        top_p=0.95,  # Nucleus sampling
    )
    # Decode and split by line
    recipe = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return recipe

# Step 4: Main Function
if __name__ == "__main__":
    # Load RecipeNLG dataset
    recipe_nlg_dataset = load_recipenlg()

    # Fine-tune the model
    fine_tuned_model_dir = fine_tune_model(recipe_nlg_dataset)
    print(fine_tuned_model_dir)

    # Example: Generate a recipe
    user_prompt1 = "Title: Chicken and Broccoli Rice Bowl, Ingredients: rice, chicken, broccoli, soy sauce, garlic, onion, olive oil"
    user_prompt2 = "Title: Aglio Olio, Ingredients: pasta, olive oil, garlic, chili"
    user_prompt3 = "Title: Caesar Salad, Ingredients: olive oil, lettuce, tomato, onion, cheese"
    generated_recipe1 = generate_recipe(fine_tuned_model_dir, user_prompt1)
    generated_recipe2 = generate_recipe(fine_tuned_model_dir, user_prompt2)
    generated_recipe3 = generate_recipe(fine_tuned_model_dir, user_prompt3)
    print("\nGenerated Recipe 1:\n")
    print(generated_recipe1)
    print("\nGenerated Recipe 2:\n")
    print(generated_recipe2)
    print("\nGenerated Recipe 3:\n")
    print(generated_recipe3)

Loading RecipeNLG dataset...
Dataset loaded and reduced: DatasetDict({
    train: Dataset({
        features: ['id', 'link', 'source', 'ner', 'text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['id', 'link', 'source', 'ner', 'text'],
        num_rows: 2000
    })
})
Fine-tuning gpt2 on RecipeNLG...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Tokenizing dataset...


Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-6a92c1d15ee3>:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 6.81 MiB is free. Process 16357 has 39.55 GiB memory in use. Of the allocated memory 38.92 GiB is allocated by PyTorch, and 125.21 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
user_prompt1 = "Title: Chicken and Broccoli Rice Bowl, Ingredients: rice, chicken, broccoli, soy sauce, garlic, onion, olive oil"
user_prompt2 = "Title: Aglio Olio, Ingredients: pasta, olive oil, garlic, chili"
user_prompt3 = "Title: Caesar Salad, Ingredients: olive oil, lettuce, tomato, onion, cheese"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
